# **Trained weight G-drive download link**

https://drive.google.com/file/d/13P9TcgBUOT_t5Bfz61LEwn-waJd1jd-Q/view?usp=sharing, https://drive.google.com/file/d/1GPuADn0-OwMfD9ZMULC0yAtE53OIDf3m/view?usp=sharing, https://drive.google.com/file/d/1HS_dlen1cfkPc1QEIArZcSRxLthEEQiy/view?usp=sharing, https://drive.google.com/file/d/1s6zlMXQG0r_384hloDrByj39lklvUP8Z/view?usp=sharing

# **U-net**

In [1]:
# load your g-dirve if you use google colab
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:

import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_probability as tfp
import math
tfd=tfp.distributions
import tensorflow_datasets as tfds
print(tf.__version__)
print(tfp.__version__)
print(tfds.__version__)

from tensorflow.keras import Model,Sequential
from tensorflow.keras.layers import Conv2D,Conv2DTranspose,Dense,BatchNormalization,Flatten,Reshape
from tensorflow.keras.losses import MeanSquaredError
#preprocess data
# mnist_train = tfds.load(name="mnist", split=tfds.Split.TRAIN)
def cast(data):
  return tf.cast(data,tf.float32)
(x_train, y_train), (x_test, y_test)=tf.keras.datasets.cifar10.load_data()
x_train=x_train/255
x_train=(x_train-0.5)*2
print(x_train.min(),x_train.max())



def ResidualBlock(width,size):
    def apply(x):
        inputs, embeddings=x
        x=inputs
        input_width = x.shape[3]
        if input_width == width:
            residual = x
        else:
            residual = tf.keras.layers.Conv2D(width, kernel_size=1)(x)
        y=keras.layers.Activation("swish")(embeddings)
        y=keras.layers.Dense(width)(y)
        x = tf.keras.layers.GroupNormalization(epsilon=1e-5)(x)
        x = tf.keras.layers.Conv2D(width, kernel_size=size, padding="same", activation=keras.activations.swish)(x)
        x = tf.keras.layers.Conv2D(width, kernel_size=size, padding="same")(x)
        x = x + y[:, None, None]
        x= tf.keras.layers.GroupNormalization(epsilon=1e-5)(x)
        x= tf.keras.layers.Activation("swish")(x)
        x = tf.keras.layers.Conv2D(width, kernel_size=size, padding="same")(x)
        x = tf.keras.layers.Add()([x, residual])
        return x

    return apply


class PaddedConv2D(keras.layers.Layer):
    def __init__(self, channels, kernel_size, padding=0, stride=1):
        super().__init__()
        self.padding2d = keras.layers.ZeroPadding2D((padding, padding))
        self.conv2d = keras.layers.Conv2D(
            channels, kernel_size, strides=(stride, stride)
        )

    def call(self, x):
        x = self.padding2d(x)
        return self.conv2d(x)

class AttentionBlock(keras.layers.Layer):
    def __init__(self, channels):
        super().__init__()
        self.norm = tf.keras.layers.GroupNormalization(epsilon=1e-5)
        self.q = PaddedConv2D(channels, 1)
        self.k = PaddedConv2D(channels, 1)
        self.v = PaddedConv2D(channels, 1)
        self.proj_out = PaddedConv2D(channels, 1)

    def call(self, x):
        h_ = self.norm(x)
        q, k, v = self.q(h_), self.k(h_), self.v(h_)

        # Compute attention
        b, h, w, c = q.shape
        q = tf.reshape(q, (-1, h * w, c))  # b,hw,c
        k = keras.layers.Permute((3, 1, 2))(k)
        k = tf.reshape(k, (-1, c, h * w))  # b,c,hw
        w_ = q @ k
        w_ = w_ * (c ** (-0.5))
        w_ = keras.activations.softmax(w_)

        # Attend to values
        v = keras.layers.Permute((3, 1, 2))(v)
        v = tf.reshape(v, (-1, c, h * w))
        w_ = keras.layers.Permute((2, 1))(w_)
        h_ = v @ w_
        h_ = keras.layers.Permute((2, 1))(h_)
        h_ = tf.reshape(h_, (-1, h, w, c))
        return x + self.proj_out(h_)




2.15.0
0.23.0
4.9.4
170498071/170498071 [==============================] - 3s 0us/step
-1.0 1.0


In [9]:
rtime_embed=keras.Input(shape=(160))
noisy_images = keras.Input(shape=(32, 32, 3))
abstract_images = keras.Input(shape=(32, 32, 3))


time_embed=keras.Input(shape=(160))
noisy_images = keras.Input(shape=(32, 32, 3))

skips=[]
ls=[4,6,8,10,12]

x=tf.keras.layers.Concatenate()([noisy_images,abstract_images])
x = tf.keras.layers.Conv2D(32*ls[0], kernel_size=3,padding="same")(x)
x=ResidualBlock(32*ls[0],4)([x,time_embed])
skips.append(x)#32

x=ResidualBlock(32*ls[1],4)([x,time_embed])
x=AttentionBlock(32*ls[1])(x)
x = tf.keras.layers.AveragePooling2D(pool_size=2)(x)
skips.append(x)#16

x=ResidualBlock(32*ls[2],4)([x,time_embed])
x=AttentionBlock(32*ls[2])(x)
x = tf.keras.layers.AveragePooling2D(pool_size=2)(x)
skips.append(x)#8

x=ResidualBlock(32*ls[3],4)([x,time_embed])
x=AttentionBlock(32*ls[3])(x)
x = tf.keras.layers.AveragePooling2D(pool_size=2)(x)
skips.append(x)#4

x=ResidualBlock(32*ls[4],4)([x,time_embed])
x=AttentionBlock(32*ls[4])(x)
x = tf.keras.layers.AveragePooling2D(pool_size=2)(x)
skips.append(x)#2


x = tf.keras.layers.Conv2D(32*32, kernel_size=1, padding="same", strides=2,activation='swish')(x)#1
embed=keras.Input(shape=(2048))
y=tf.keras.layers.Reshape((1,1,2048))(embed)
x = tf.keras.layers.Concatenate()([x, y])

x = tf.keras.layers.UpSampling2D(size=2, interpolation="bilinear")(x)#2

x = tf.keras.layers.Conv2D(32*ls[4], kernel_size=2, padding="same",activation=keras.activations.swish)(x)
x=ResidualBlock(32*ls[4],4)([x,time_embed])
x=AttentionBlock(32*ls[4])(x)

x = tf.keras.layers.Concatenate()([x, skips.pop()])

# y = tf.keras.layers.UpSampling2D(size=2, interpolation="bilinear")(y)
# y = tf.keras.layers.Conv2D(32*ls[4], kernel_size=2, padding="same",activation=keras.activations.swish)(y)
# y=AttentionBlock(32*ls[4])(x)


x=ResidualBlock(32*ls[3],4)([x,time_embed])
x=AttentionBlock(32*ls[3])(x)
x=ResidualBlock(32*ls[3],4)([x,time_embed])
x = tf.keras.layers.UpSampling2D(size=2, interpolation="bilinear")(x)#4

x = tf.keras.layers.Concatenate()([x, skips.pop()])
x=ResidualBlock(32*ls[2],4)([x,time_embed])
x=AttentionBlock(32*ls[2])(x)
x=ResidualBlock(32*ls[2],4)([x,time_embed])
x = tf.keras.layers.UpSampling2D(size=2, interpolation="bilinear")(x)#8

x = tf.keras.layers.Concatenate()([x, skips.pop()])
x=ResidualBlock(32*ls[2],4)([x,time_embed])
x=AttentionBlock(32*ls[2])(x)
x = tf.keras.layers.UpSampling2D(size=2, interpolation="bilinear")(x)#16
x=ResidualBlock(32*ls[1],4)([x,time_embed])

x = tf.keras.layers.Concatenate()([x, skips.pop()])
x=ResidualBlock(32*ls[1],4)([x,time_embed])
x=AttentionBlock(32*ls[1])(x)
x = tf.keras.layers.UpSampling2D(size=2, interpolation="bilinear")(x)#32
x=ResidualBlock(32*ls[1],4)([x,time_embed])

x = tf.keras.layers.Concatenate()([x, skips.pop()])
x=ResidualBlock(32*ls[0],4)([x,time_embed])
x=AttentionBlock(32*ls[0])(x)
# x = tf.keras.layers.UpSampling2D(size=2, interpolation="bilinear")(x)

x = tf.keras.layers.Conv2D(32, kernel_size=4, padding="same")(x)
x = tf.keras.layers.GroupNormalization(epsilon=1e-5)(x)
x = tf.keras.layers.Activation('swish')(x)
x = tf.keras.layers.Conv2D(3, kernel_size=3,padding="same")(x)#, kernel_initializer="zeros"
# x = tf.keras.layers.Activation('swish')(x)

model=keras.Model([noisy_images,abstract_images,embed,time_embed], x)
model.summary()



Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_6 (InputLayer)        [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 input_4 (InputLayer)        [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 concatenate (Concatenate)   (None, 32, 32, 6)            0         ['input_6[0][0]',             
                                                                     'input_4[0][0]']             
                                                                                                  
 conv2d_16 (Conv2D)          (None, 32, 32, 128)          7040      ['concatenate[0][0]']   

# **Generator blur image conditioning**

In [3]:
def ResidualBlock_noT(width,size):
    def apply(x):
        input_width = x.shape[3]
        if input_width == width:
            residual = x
        else:
            residual = tf.keras.layers.Conv2D(width, kernel_size=1)(x)
        x = tf.keras.layers.GroupNormalization(epsilon=1e-5)(x)
        x = tf.keras.layers.Conv2D(width, kernel_size=size, padding="same", activation=keras.activations.swish)(x)
        x = tf.keras.layers.Conv2D(width, kernel_size=size, padding="same")(x)
        x = tf.keras.layers.Add()([x, residual])
        return x

    return apply

embed=keras.Input(shape=(2048))
y=tf.keras.layers.Reshape((1,1,2048))(embed)
y = tf.keras.layers.Conv2DTranspose(512, kernel_size=2, padding="valid", activation=keras.activations.swish)(y)
y = tf.keras.layers.Conv2DTranspose(256, kernel_size=2, strides=2,padding="valid", activation=keras.activations.swish)(y)
y=ResidualBlock_noT(256,4)(y)
y=AttentionBlock(256)(y)
y = tf.keras.layers.Conv2DTranspose(128, kernel_size=2, strides=2,padding="valid", activation=keras.activations.swish)(y)
y=AttentionBlock(128)(y)
y = tf.keras.layers.Conv2DTranspose(128, kernel_size=2, strides=2,padding="valid", activation=keras.activations.swish)(y)
y=AttentionBlock(128)(y)
y = tf.keras.layers.Conv2DTranspose(128, kernel_size=2, strides=2,padding="valid", activation=keras.activations.swish)(y)
y = tf.keras.layers.Conv2D(32, kernel_size=4, padding="same", activation=keras.activations.swish)(y)
y = tf.keras.layers.GroupNormalization(epsilon=1e-5)(y)
y = tf.keras.layers.Activation('swish')(y)
y = tf.keras.layers.Conv2D(3, kernel_size=3,padding="same")(y)
generator=keras.Model(embed, y)
generator.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 2048)]               0         []                            
                                                                                                  
 reshape (Reshape)           (None, 1, 1, 2048)           0         ['input_1[0][0]']             
                                                                                                  
 conv2d_transpose (Conv2DTr  (None, 2, 2, 512)            4194816   ['reshape[0][0]']             
 anspose)                                                                                         
                                                                                                  
 conv2d_transpose_1 (Conv2D  (None, 4, 4, 256)            524544    ['conv2d_transpose[0][0]']

# **Diffusion coefficients**

In [4]:
#
dist_U=tfd.Normal(0.0,1.0)
nums_steps=100
beta=np.exp(np.linspace(-5.5,-2.50,nums_steps))
alpha_f=np.zeros(nums_steps+1)
for i in range(1,nums_steps+1):
    z_t=[]
    for k in range(i):
        z=[]
        for j in range(i-k):
            z.append(beta[j+k])
        z_t.append(z)

    kkk=np.ones(len(z_t))
    for d in range(len(z_t)):
        for q in range((len(z_t[d]))):
            if q==0:
                kkk[d]=kkk[d]*z_t[d][q]
            else:
                kkk[d]=kkk[d]*(1-z_t[d][q])
    alpha_f[i]=np.sum(kkk)

alpha_f=alpha_f[1:]
print(len(alpha_f))
print(alpha_f)



100
[0.00408677 0.00828206 0.01258822 0.01700758 0.02154253 0.02619544
 0.0309687  0.03586469 0.04088583 0.04603449 0.05131308 0.05672396
 0.06226951 0.06795207 0.07377397 0.0797375  0.08584494 0.09209851
 0.09850038 0.10505269 0.1117575  0.11861682 0.12563258 0.13280663
 0.14014073 0.14763654 0.15529562 0.16311941 0.17110921 0.17926621
 0.18759143 0.19608575 0.20474986 0.21358429 0.22258937 0.23176522
 0.24111174 0.25062862 0.26031528 0.2701709  0.2801944  0.29038438
 0.30073919 0.31125683 0.321935   0.33277106 0.34376202 0.35490451
 0.36619481 0.37762879 0.38920195 0.40090935 0.41274564 0.42470505
 0.43678137 0.44896794 0.46125766 0.47364296 0.48611582 0.49866778
 0.5112899  0.5239728  0.53670663 0.54948114 0.5622856  0.5751089
 0.5879395  0.60076549 0.61357457 0.62635415 0.63909127 0.65177272
 0.66438503 0.67691453 0.68934737 0.70166957 0.71386708 0.72592582
 0.73783173 0.74957084 0.76112931 0.77249351 0.78365007 0.79458594
 0.80528849 0.81574555 0.82594545 0.83587716 0.84553028 0.8

# **Time condition**

In [5]:
def get_timestep_embedding(timestep, dim=160, max_period=nums_steps):
    half = dim // 2
    range = tf.cast(np.arange(0, half), "float32")
    freqs = tf.exp(-np.log(max_period) * range / half)
    args = tf.convert_to_tensor([timestep], dtype="float32") * freqs
    embedding = np.concatenate([np.cos(args), np.sin(args)], 0)
    # embedding = tf.reshape(embedding, [1, -1])
    return embedding

nums_steps_select=100

time_latents=[]
for i in range(nums_steps_select):
    time_latents.append(get_timestep_embedding(i))
time_latents=np.asarray(time_latents)

alpha_f=alpha_f[:nums_steps_select]
print('length of time latent',len(time_latents))




length of time latent 100


# **GMM condition**

pre-made by designed module for Cifar10 training set images. Download from repository

In [6]:
embed_latents=np.load('xxxxxxxxx/latent_2048.npy')

resample from distribution $Z_i$

In [7]:
numbers_of_random_samples=25
types=np.asarray([[0,0,0,1],[0,0,1,0],[0,1,0,0],[1,0,0,0]])
dist_cata=tfd.Categorical(probs=[0.25,0.25,0.25,0.25])
dist_gmm=tfd.MixtureSameFamily(
      mixture_distribution=tfd.Categorical(probs=cast([1/2,1/2])),
      components_distribution=tfd.Normal(
          loc=cast([3.0,6.0]),
          scale=cast([0.5,.5])))

def sample_gmm_z_i(dex):
  random_latent_samples=[]
  rand_latent_inp=np.zeros((numbers_of_random_samples+1,2048))
  bed_input=embed_latents[dex]
  rand_latent_inp[-1]=bed_input
  for j in range(512):
    if np.sum(bed_input[j*4:j*4+4])>0:
      for i in range(numbers_of_random_samples):
        rand_latent_inp[i][j*4:j*4+4]=types[dist_cata.sample(1).numpy()[0]]*dist_gmm.sample(1)
  return rand_latent_inp

# **Load_weights**

In [10]:
model.load_weights('xxxxxxxxx/testCifar_step_100_denoise_with_img_latent_20wstep_v3.h5')#download from repository
generator.load_weights('xxxxxxxxxx/Generators_diffusion_cifar.h5')#download from repository

In [11]:

@tf.function
def model_out(inputs_all):
    generated_images = model(inputs_all, training=False)
    return generated_images



# **Generate Images (Cifar10)**

In [12]:

def show_diffusion_step():
  test_index=np.random.randint(0,len(x_train))
  output2=[]
  bed_input=embed_latents[test_index][np.newaxis,...]
  gen_input=np.clip(generator(bed_input).numpy(),-1.0,1.0)
  noise_img=dist_U.sample((1,32,32,3))
  for i in range(nums_steps_select-1):
    denoised_img=model_out([noise_img,gen_input,bed_input,time_latents[i][tf.newaxis,...]])
    img_at_t=np.clip((noise_img-denoised_img*(1-alpha_f[i]))[0]/alpha_f[i],-1.0,1.0)
    output2.append(img_at_t)
    noise_img=np.asarray(noise_img).copy()
    noise_img=tf.random.normal((1,32,32,3))*(1-alpha_f[i+1])+img_at_t*alpha_f[i+1]
  output2.append(x_train[test_index])
  output2=np.asarray(output2)
  output2=output2*0.5+0.5
  nrow=10
  ncol=10
  fig, axs = plt.subplots(nrow, ncol,figsize=(1*ncol,1*nrow))
  z=0
  for j in range(nrow):
    for i in range(ncol):
      axs[j,i].imshow(output2[z])
      axs[j,i].set_axis_off()
      z+=1
      plt.tight_layout()
  plt.show()
  return
for i in range(5):
  show_diffusion_step()

Output hidden; open in https://colab.research.google.com to view.

# **Generate Images (random sample from $Z_i$)**

In [13]:

def gen_images(img_latents_inp,embed_latents_inp):
  output2=[]
  gen_input=img_latents_inp[np.newaxis,...]
  bed_input=embed_latents_inp[np.newaxis,...]
  noise_img=dist_U.sample((1,32,32,3))
  for i in range(nums_steps_select-1):
    denoised_img=model_out([noise_img,gen_input,bed_input,time_latents[i][tf.newaxis,...]])
    img_at_t=np.clip((noise_img-denoised_img*(1-alpha_f[i]))[0]/alpha_f[i],-1.0,1.0)
    output2.append(img_at_t)
    noise_img=np.asarray(noise_img).copy()
    noise_img=tf.random.normal((1,32,32,3))*(1-alpha_f[i+1])+img_at_t*alpha_f[i+1]
  output2=np.asarray(output2)
  output2=output2*0.5+0.5
  return output2


In [14]:
def gen_images_from_Zi(dex):
  random_generations=[]
  rand_latent_inp=sample_gmm_z_i(dex)
  rand_latent_img=generator(rand_latent_inp).numpy()
  rand_latent_img=np.clip(rand_latent_img,-1.0,1.0)
  for i in range(len(rand_latent_inp)):
    random_generations.append(gen_images(rand_latent_img[i],rand_latent_inp[i]))
  return random_generations


In [ ]:
index_list=[]
output_rand=[]
for i in range(100):
  dex_inp=np.random.randint(0,50000)
  index_list.append(dex_inp)
  output_rand.append(gen_images_from_Zi(dex_inp))
  print('step:',i,'random index:{}'.format(dex_inp))


In [23]:
nrow=1
ncol=15

for j in range(64):
  print('Samples from distribution Z_{}'.format(index_list[j]))
  fig, axs = plt.subplots(nrow, ncol,figsize=(1*ncol,1*nrow))
  for i in range(ncol):
    axs[i].imshow(output_rand[j][np.random.randint(0,25)][-3])
    axs[i].set_axis_off()
  plt.tight_layout()
  plt.show()

Output hidden; open in https://colab.research.google.com to view.

# **Distribution of sample mean($\overline{\text{FID}},\overline{\text{IS}}$) in Cifar10, Tensorflow 2. Size of image set=1000**

$\overline{\text{IS}}$, size of smaple imageset=1000

In [ ]:
# calculate inception score for cifar-10 in Keras
from math import floor
from numpy import ones
from numpy import expand_dims
from numpy import log
from numpy import mean
from numpy import std
from numpy import exp
from numpy.random import shuffle
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.datasets import cifar10
from skimage.transform import resize
from numpy import asarray

# scale an array of images to a new size
def scale_images(images, new_shape):
	images_list = list()
	for image in images:
		# resize with nearest neighbor interpolation
		new_image = resize(image, new_shape, 0)
		# store
		images_list.append(new_image)
	return asarray(images_list)

# assumes images have any shape and pixels in [0,255]


In [ ]:
# load cifar10 images
(images1, _), (images2, _) = cifar10.load_data()
# shuffle images
shuffle(images1)
print('loaded', images1.shape)

170498071/170498071 [==============================] - 2s 0us/step
loaded (50000, 32, 32, 3)


In [ ]:

def calculate_inception_score(images, n_split=10, eps=1E-16):
  model = InceptionV3()
  scores = list()
  n_part = floor(images.shape[0] / n_split)
  for i in range(n_split):
    ix_start, ix_end = i * n_part, (i+1) * n_part
    subset = images[ix_start:ix_end]
    subset = subset.astype('float32')
    subset = scale_images(subset, (299,299,3))
    subset = preprocess_input(subset)
    p_yx = model.predict(subset)
    p_y = expand_dims(p_yx.mean(axis=0), 0)
    kl_d = p_yx * (log(p_yx + eps) - log(p_y + eps))
    sum_kl_d = kl_d.sum(axis=1)
    avg_kl_d = mean(sum_kl_d)
    is_score = exp(avg_kl_d)
    scores.append(is_score)
    print(is_score)
  is_avg, is_std = mean(scores), std(scores)
  return is_avg, is_std

In [ ]:
# calculate inception score
is_avg, is_std = calculate_inception_score(images1,50)
print('score', is_avg, is_std)

170498071/170498071 [==============================] - 11s 0us/step
loaded (50000, 32, 32, 3)
32/32 [==============================] - 40s 1s/step
11.309804
32/32 [==============================] - 38s 1s/step
11.021277
32/32 [==============================] - 39s 1s/step
11.743633
32/32 [==============================] - 39s 1s/step
11.085611
32/32 [==============================] - 39s 1s/step
10.863561
32/32 [==============================] - 39s 1s/step
10.546839
32/32 [==============================] - 39s 1s/step
11.197749
32/32 [==============================] - 39s 1s/step
11.332624
32/32 [==============================] - 40s 1s/step
11.542571
32/32 [==============================] - 39s 1s/step
10.818204
32/32 [==============================] - 39s 1s/step
10.86397
32/32 [==============================] - 44s 1s/step
10.374776
32/32 [==============================] - 42s 1s/step
11.019383
32/32 [==============================] - 44s 1s/step
10.526156
32/32 [==================

$\overline{\text{FID}}$, size of smaple imageset=1000

In [ ]:
# example of calculating the frechet inception distance in Keras for cifar10
import numpy as np
import tensorflow as tf
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy import asarray
from numpy.random import shuffle
from scipy.linalg import sqrtm
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.datasets.mnist import load_data
from skimage.transform import resize
from keras.datasets import cifar10

# scale an array of images to a new size
def scale_images(images, new_shape):
	images_list = list()
	for image in images:
		# resize with nearest neighbor interpolation
		new_image = resize(image, new_shape, 0)
		# store
		images_list.append(new_image)
	return asarray(images_list)



model = InceptionV3(include_top=False, pooling='avg', input_shape=(299,299,3))


87910968/87910968 [==============================] - 0s 0us/step


In [ ]:
(images1, _), (images2, _) = cifar10.load_data()
shuffle(images1)
# images1 = images1[:10000]
print('Loaded', images1.shape, images2.shape)
# convert integer to floating point values
images1 = images1.astype('float32')
images2 = images2.astype('float32')
# resize images
print('Scaled', images1.shape, images2.shape)


Loaded (50000, 32, 32, 3) (10000, 32, 32, 3)
Scaled (50000, 32, 32, 3) (10000, 32, 32, 3)


In [ ]:
#Numpy function (not applied)
# def _symmetric_matrix_square_root_np(mat, eps=1e-10):
#   u, s, v = np.linalg.svd(mat)
#   si=np.where(np.less(s, 1e-10), s, np.sqrt(s))

#   return np.dot(np.dot(u,np.diag(si)),v)

# def trace_sqrt_product_np(sigma, sigma_v):

#   sqrt_sigma = _symmetric_matrix_square_root_np(sigma)

#   sqrt_a_sigmav_a = np.dot(sqrt_sigma, np.dot(sigma_v, sqrt_sigma))

#   return trace(_symmetric_matrix_square_root_np(sqrt_a_sigmav_a))



In [ ]:

def _symmetric_matrix_square_root(mat, eps=1e-10):

  # Unlike numpy, tensorflow's return order is (s, u, v)
  s, u, v = tf.linalg.svd(mat)
  # sqrt is unstable around 0, just use 0 in such case
  si = tf.compat.v1.where(tf.less(s, eps), s, tf.sqrt(s))
  # Note that the v returned by Tensorflow is v = V
  # (when referencing the equation A = U S V^T)
  # This is unlike Numpy which returns v = V^T
  return tf.matmul(tf.matmul(u, tf.linalg.tensor_diag(si)), v, transpose_b=True)

def trace_sqrt_product(sigma, sigma_v):

  sqrt_sigma = _symmetric_matrix_square_root(sigma)

  sqrt_a_sigmav_a = tf.matmul(sqrt_sigma, tf.matmul(sigma_v, sqrt_sigma))

  return tf.linalg.trace(_symmetric_matrix_square_root(sqrt_a_sigmav_a))

def find_mean_var_tf(model,images):
  act = model.predict(images)
  mu, sigma = act.mean(axis=0), cov(act, rowvar=False)
  return mu, sigma

def cast(data):
  return tf.cast(data,tf.float32)

@tf.function
def find_fid_tf(model,mu1, sigma1,mu2, sigma2 ):
  mu1=cast(mu1)
  mu2=cast(mu2)
  sigma1=cast(sigma1)
  sigma2=cast(sigma2)
  ssdiff = tf.reduce_sum((mu1 - mu2)**2.0)
  covmean = trace_sqrt_product(sigma1,sigma2)
  fid = ssdiff + tf.linalg.trace(sigma1 + sigma2) - 2.0 * covmean
  return fid

In [ ]:
images_total=np.concatenate((images1,images2))
images_total.shape

(60000, 32, 32, 3)

samples of mean and variance of distribution $N_1(\mu_1,\Sigma_1)$

In [ ]:
m1_s1=[]
for i in range(30):
  images_target=images_total[np.random.randint(0,60000,1000)]
  print(images_target.shape)
  images_target = scale_images(images_target, (299,299,3))
  images_target = preprocess_input(images_target)
  m1,s1=find_mean_var_tf(model,images_target)
  # print('mean:',m1,'std:',s1)
  m1_s1.append([m1,s1])
m1_s1=np.asarray(m1_s1)

(1000, 32, 32, 3)
32/32 [==============================] - 59s 2s/step
(1000, 32, 32, 3)
32/32 [==============================] - 57s 2s/step
(1000, 32, 32, 3)
32/32 [==============================] - 57s 2s/step
(1000, 32, 32, 3)
32/32 [==============================] - 57s 2s/step
(1000, 32, 32, 3)
32/32 [==============================] - 56s 2s/step
(1000, 32, 32, 3)
32/32 [==============================] - 57s 2s/step
(1000, 32, 32, 3)
32/32 [==============================] - 57s 2s/step
(1000, 32, 32, 3)
32/32 [==============================] - 58s 2s/step
(1000, 32, 32, 3)
32/32 [==============================] - 56s 2s/step
(1000, 32, 32, 3)
30/32 [===========================>..] - ETA: 3s

In [ ]:
m1=[]
s1=[]
for i in range(33):
  m1.append(m1_s1[i][0])
  s1.append(m1_s1[i][1])
m1=np.asarray(m1)
s1=np.asarray(s1)
print(m1.shape,s1.shape)

(33, 2048) (33, 2048, 2048)


samples of mean and variance of distribution $N_2(\mu_2,\Sigma_2)$ and calculate $\overline{\text{FID}}$ with samples from $N_1$

In [ ]:
fid=[]
for i in range(5):
  images_test=images_total[np.random.randint(0,60000,1000)]
  images_test = scale_images(images_test, (299,299,3))
  images_test = preprocess_input(images_test)
  m2,s2=find_mean_var_tf(model,images_test)
  for j in range(33):
    val=find_fid_tf(model,m1[j],s1[j],m2,s2)
    print(val)
    fid.append(val)


32/32 [==============================] - 57s 2s/step
tf.Tensor(53.524963, shape=(), dtype=float32)
tf.Tensor(53.166534, shape=(), dtype=float32)
tf.Tensor(53.658386, shape=(), dtype=float32)
tf.Tensor(53.776215, shape=(), dtype=float32)
tf.Tensor(52.90146, shape=(), dtype=float32)
tf.Tensor(53.938538, shape=(), dtype=float32)
tf.Tensor(53.700623, shape=(), dtype=float32)
tf.Tensor(53.17914, shape=(), dtype=float32)
tf.Tensor(53.075653, shape=(), dtype=float32)
tf.Tensor(53.39325, shape=(), dtype=float32)
tf.Tensor(54.855927, shape=(), dtype=float32)
tf.Tensor(53.450775, shape=(), dtype=float32)
tf.Tensor(54.200867, shape=(), dtype=float32)
tf.Tensor(54.165894, shape=(), dtype=float32)
tf.Tensor(52.802277, shape=(), dtype=float32)
tf.Tensor(53.367004, shape=(), dtype=float32)
tf.Tensor(53.15927, shape=(), dtype=float32)
tf.Tensor(53.394226, shape=(), dtype=float32)
tf.Tensor(53.36609, shape=(), dtype=float32)
tf.Tensor(53.891724, shape=(), dtype=float32)
tf.Tensor(53.928955, shape=(), d

In [ ]:
fid=np.asarray(fid)
print('FID score of Dataset    ', ' Mean:    ', fid.mean(),'Std:   ', fid.std())

FID score of Dataset      Mean:     53.629745 Std:    0.5881507
